In [1]:
# Import libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup

C:\Users\Chris\AppData\Local\Temp\ipykernel_38248\894987728.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Functions

In [12]:
def scrape_page(url, scrape_func):
    """Scrape the CCBC website for a given url and scrape_func

    Args:
        url (str): The url to scrape
        scrape_func (function): The function to use to scrape the table

    Returns:
        pd.DataFrame: The dataframe containing the scraped data
    """    
    
    # Get the page
    page = requests.get(url)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the select element with the id seasonid
    select = soup.find('select', {'id': 'seasonid'})
    # Get the options from the select element (this is the list of years)
    options = select.find_all('option')
    # Create a dictionary of the years (seasons) with the season_id as the key and the season name as the value
    seasons = {option['value']: option.text for option in options}
    print(seasons)
    
    # For season in season_ids scrape the table and append the results to a dataframe using the scrape_func
    df = pd.DataFrame()
    for season in seasons.keys():
        temp_df = scrape_func(season)
        # append the new row to the DataFrame
        df = pd.concat([df, temp_df])

    # Create a new variable called season that uses the season_id column to look up the season name in the seasons dictionary
    df['season'] = df['season_id'].map(seasons)

    # Drop the season_id column
    df.drop('season_id', axis=1, inplace=True)
    
    # Return the dataframe
    return df


def scrape_table(season_id, table):
    """Scrape the batting table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the batting table for

    Returns:
        pd.DataFrame: The batting table for the given season_id
    """    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view={table}'
    
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#bat_first > table:nth-child(1)')
    
    # Check if the table is empty
    if not table:
        print(f'No table found for season_id {season_id}')
        return None
    
    # Get the headers (th) from the table
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []
    
    # Get the rows (tr) from the table
    for tr in table[0].find_all('tr'):
        # Get the data (td) from the row
        row = [td.text.strip() for td in tr.find_all('td')]
        # Check if the row is empty if not append the season_id to the row and append the row to the rows list
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)



In [9]:
# Declare the URL
url = 'http://pointstreak.com/baseball/stats.html?leagueid=160&seasonid=186&view=teambatting'

path = 'http://pointstreak.com/baseball/stats.html?leagueid=160&seasonid={season_id}&view=teambatting'

# Call the scrape_page function and pass in the url
batting_df = scrape_page(url, scrape_table)

# Preview the dataframe
batting_df.head()

# Save as a csv file
batting_df.to_csv('ccbc_batting.csv', index=False)

NameError: name 'season_id' is not defined

In [7]:
# Preview the dataframe
batting_df.head()

,Team,G,AB,R,H,2B,3B,HR,RBI,TB,BB,season
0,PBA,5,181,40,63,8,1,0,32,73,20,2023 CCBC Champs
1,UFV,2,69,6,22,2,1,1,5,29,3,2023 CCBC Champs
2,UC,4,142,37,43,8,2,0,22,55,26,2023 CCBC Champs
3,VC,5,189,41,57,13,1,1,28,75,19,2023 CCBC Champs
4,OC,4,128,23,35,4,0,1,18,42,22,2023 CCBC Champs
